In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap


import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')

dfCurAdmitted = pd.read_csv(latestdir+'\\Gennembruds_DB\\01_indlagte_pr_vaccinationsstatus.csv',delimiter=';',encoding='latin1')
df = pd.read_csv(latestdir+'\\Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv',delimiter=';',encoding='latin1')

df['Nyindlagte pr. 100.000'] = pd.to_numeric(df['Nyindlagte pr. 100.000'].str.replace(',','.'))
dfCurAdmitted

,Vaccinationsstatus,Antal indlagte,Befolkningstørrelse,Indlagte pr. 100.000
0,Forventet fuld effekt,335,4479055,"7,48"
1,Ikke vaccineret,171,519778,"32,90"
2,Påbegyndt vaccinationsforløb,19,132645,"14,32"


In [3]:
# curCount.values[0]
# df
# df[df.Vaccinationsstatus == 'Ikke vaccineret') & df.].loc[550:]
# df[(df.Vaccinationsstatus == 'Ikke vaccineret') & (df.Aldersgruppe == '80+')].iloc[-15:]
# print(df.Aldersgruppe.unique())
df[df.Aldersgruppe == '80+'].tail(3)
# df[df.Aldersgruppe == '80+']
df

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
0,2020-W52,12-15,Forventet fuld effekt,0,0,0.00
1,2020-W52,12-15,Ikke vaccineret,2,283610,0.71
2,2020-W52,12-15,Påbegyndt vaccinationsforløb,0,0,0.00
3,2020-W52,16-19,Forventet fuld effekt,0,0,0.00
4,2020-W52,16-19,Ikke vaccineret,7,278165,2.52
5,2020-W52,16-19,Påbegyndt vaccinationsforløb,0,0,0.00
6,2020-W52,20-39,Forventet fuld effekt,0,0,0.00
7,2020-W52,20-39,Ikke vaccineret,99,1485292,6.67
8,2020-W52,20-39,Påbegyndt vaccinationsforløb,0,6,0.00
9,2020-W52,40-64,Forventet fuld effekt,0,0,0.00


In [4]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfVacc = dfShort[dfShort.Vaccinationsstatus == 'Forventet fuld effekt']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])

In [5]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]
weekLabelsSpace = ['    ' + x for x in weekLabels ]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfUnvacc.groupby('Aldersgruppe'))
# thisdf = list(dfUnvacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])
agedf

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
364,2021-W20,80+,Ikke vaccineret,1,13318,7.51
382,2021-W21,80+,Ikke vaccineret,0,12660,0.00
400,2021-W22,80+,Ikke vaccineret,1,12078,8.28
418,2021-W23,80+,Ikke vaccineret,1,11685,8.56
433,2021-W24,80+,Ikke vaccineret,1,11364,8.80
463,2021-W26,80+,Ikke vaccineret,0,10947,0.00
481,2021-W27,80+,Ikke vaccineret,1,10789,9.27
493,2021-W28,80+,Ikke vaccineret,0,10642,0.00
511,2021-W29,80+,Ikke vaccineret,1,10502,9.52
529,2021-W30,80+,Ikke vaccineret,0,10393,0.00


In [6]:
# dfVacc
# list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# fig,ax1 = plt.subplots()
# ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'],'*-')¨
fullMatVaccInci

array([[ 0.  ,  0.  ,  1.53,  0.  ,  0.33,  0.  ],
       [ 0.  ,  0.  ,  1.33,  0.53,  0.67,  1.2 ],
       [ 0.  ,  0.  ,  0.  ,  1.39,  0.35,  1.98],
       [ 0.  ,  0.  ,  0.  ,  1.24,  0.49,  0.79],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.3 ,  1.18],
       [ 0.  ,  0.  ,  0.  ,  0.32,  0.41,  0.  ],
       [ 0.  , 19.65,  0.  ,  0.  ,  0.  ,  0.39],
       [ 0.  ,  0.  ,  0.  ,  0.2 ,  0.48,  0.  ],
       [ 0.  ,  0.  ,  1.83,  0.28,  0.71,  1.16],
       [ 0.  ,  6.18,  0.59,  0.  ,  0.47,  2.71],
       [ 0.  ,  0.  ,  1.08,  0.59,  0.82,  1.54],
       [ 0.  ,  0.  ,  0.41,  0.87,  0.94,  5.01],
       [ 0.  ,  0.62,  0.3 ,  0.59,  1.4 ,  3.08],
       [ 0.  ,  0.  ,  0.44,  1.  ,  2.68,  4.23],
       [ 0.  ,  0.49,  0.29,  0.67,  2.8 ,  6.15],
       [ 0.  ,  0.  ,  0.23,  0.65,  1.51,  6.15],
       [ 0.  ,  0.  ,  0.31,  0.94,  1.86,  6.14],
       [ 0.  ,  0.  ,  0.29,  0.47,  1.39,  6.52],
       [ 0.  ,  0.  ,  0.28,  0.35,  1.97,  6.14],
       [ 0.6 ,  0.  ,  0.28,  0

In [7]:

levelsToShow = [0,1,2,5,10,20,50,100]
levelsToShow = [0,5,10,20,50,100]
hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
hex_colors = sns.light_palette("#4040ff", n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette('DarkBlue', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(("#ffffff","#0000ff","#000000"), n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(, n_colors=len(levelsToShow) , as_cmap=False).as_hex()

# print(hex_colors) 

new_hex_colors = []
curVal = 0
# prevVal = 0
for k in range(len(levelsToShow)-1):
    # print(k)
    
    curHexColor = hex_colors[k]
    
    nextVal = levelsToShow[k+1] 
    curVal = levelsToShow[k] 
    for n in range(nextVal-curVal):
        new_hex_colors.append(curHexColor)

        
#     print(levelsToShow[k])
    
#     print(nextVal)
#     if curVal < nextVal:
#         new_hex_colors.append(curHexColor)
#         curVal = curVal + 1
#         print(curVal)
#     # prevVal = curVal
# print(new_hex_colors)
hex_colors

['#f0f0f3', '#cdcdf5', '#aaaaf8', '#8787fa', '#6363fd', '#4040ff']

In [8]:
fig,ax1 = plt.subplots(figsize=(14,8))


# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))
cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)

SourceText = 'Rasmus K. Pedersen, PandemiX center, RUC - Datakilde: covid19.ssi.dk/overvagningsdata \n Fra dashboard-fil "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
# ax1.text(18,-0.75,SourceText,ha='right')
ax1.text(31,-1.15,SourceText,ha='right',fontsize=7)

# plt.title('Vaccinated, hospitalizations per 100.000')
plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')
plt.tight_layout()
if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Vaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig,ax1 = plt.subplots(figsize=(14,8))


# levelsToShow = [0,1,5,10,20,50]
# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# # myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))

# g = sns.heatmap(fullMatUnvaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.0f',
#                 vmax=30,
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)

# plt.title('Unvaccinated, hospitalizations per 100.000')
# plt.title('Uvaccinerede, nye indlæggelser per 100.000')
plt.title('Nye indlæggelser per 100.000 uvaccinerede i aldersgruppen')
plt.tight_layout()

SourceText = 'Rasmus K. Pedersen, PandemiX center, RUC - Datakilde: covid19.ssi.dk/overvagningsdata \n Fra dashboard-fil "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
# ax1.text(18,-0.75,SourceText,ha='right')
ax1.text(31,-1.15,SourceText,ha='right',fontsize=7)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(14,10))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)
# cbar.set_ticklabels(levelsToShow,textsize=12)
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=10)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

ax2.text(31,-2.25,SourceText,ha='right',fontsize=7)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_BothLong')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Shorten data to use - EVEN SHORTER!
firstWeek = 32
firstIndexToUse = df[df.Uge == f'2021-W{firstWeek}'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]
weekLabelsSpace = ['    ' + x for x in weekLabels]
weekLabels_eng = ['Week '+str(x) for x in weekRange]
weekLabelsSpace_eng = ['    ' + x for x in weekLabels_eng]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
            
fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount

In [12]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

ax2.text(18,-1.15,SourceText,ha='right',fontsize=7)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                # annot=True,
                # fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                # annot=True,
                # fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

ax2.text(18,-1.15,SourceText,ha='right',fontsize=7)
fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_NoText')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('New hospital-admissions per\n100,000 vaccinated in group.')
ax2.set_title('New hospital-admissions per\n100,000 unvaccinated in group.')

fig.tight_layout()

SourceText_eng = 'Rasmus K. Pedersen, PandemiX center, Roskilde University - Data-source: covid19.ssi.dk/overvagningsdata \n Filename: "Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv"'
ax2.text(18,-1.35,SourceText_eng,ha='right',fontsize=7)

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_English')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# asdf = df[(df.Aldersgruppe == '65-79') & (df.Vaccinationsstatus == 'Forventet fuld effekt')]
# asdf2 = df[(df.Aldersgruppe == '65-79') & (df.Vaccinationsstatus == 'Ikke vaccineret')]
# asdf3 = df[(df.Aldersgruppe == '65-79') & (df.Vaccinationsstatus == 'Påbegyndt vaccinationsforløb')]
# df
# vac = asdf.Befolkningstørrelse.values
# uvac = asdf2.Befolkningstørrelse.values
# semivac = asdf3.Befolkningstørrelse.values
# plt.figure()
# # plt.fill_between(np.arange(len(asdf.Uge)),vac)
# # plt.fill_between(np.arange(len(asdf.Uge)),vac+uvac)
# # plt.fill_between(np.arange(len(asdf.Uge)),vac+uvac+semivac)
# plt.plot(np.arange(len(asdf.Uge)),vac)
# plt.plot(np.arange(len(asdf.Uge)),vac+uvac)
# plt.plot(np.arange(len(asdf.Uge)),vac+uvac+semivac)
# # plt.plot(asdf.Befolkningstørrelse)
# # plt.plot(asdf2.Befolkningstørrelse)

# Try with extended labels

In [16]:
# Shorten data to use - EVEN SHORTER!
firstWeek = 32
firstIndexToUse = df[df.Uge == f'2021-W{firstWeek}'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
    fullMatUnvaccPop[:,ageIndex] = agePop

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
    fullMatVaccPop[:,ageIndex] = agePop

In [17]:
fullMatVaccPop

array([[   1942.,  160321.,  330105., 1514850.,  855895.,  259696.],
       [   7993.,  188326.,  459429., 1598392.,  857147.,  259882.],
       [  27811.,  204461.,  684227., 1652589.,  858605.,  260127.],
       [  66250.,  214281.,  872675., 1683318.,  859888.,  260355.],
       [ 102968.,  214281.,  978257., 1698135.,  860685.,  260522.],
       [ 102968.,  214281., 1036450., 1708105.,  861300.,  260621.],
       [ 102968.,  226727., 1063648., 1714150.,  861502.,  260390.],
       [ 167636.,  228851., 1082827., 1718883.,  861608.,  260096.],
       [ 176830.,  230547., 1097398., 1722647.,  861699.,  259756.],
       [ 176830.,  231751., 1107657., 1725447.,  861675.,  259379.],
       [ 186959.,  232646., 1114225., 1727406.,  861579.,  258913.],
       [ 190118.,  233321., 1119818., 1729138.,  861458.,  258408.],
       [ 192505.,  233786., 1124362., 1730456.,  861302.,  257963.],
       [ 195013.,  234276., 1128717., 1731795.,  861096.,  257433.],
       [ 197141.,  234769., 113316

In [18]:
plotdf = pd.DataFrame(fullMatVaccInci.transpose())
annodf = pd.DataFrame(fullMatVaccCount.transpose())
annodf2 = pd.DataFrame(fullMatVaccPop.transpose())
annodf3 = pd.DataFrame(fullMatVaccCount.transpose())
annodf3_eng = pd.DataFrame(fullMatVaccCount.transpose())
# display(plotdf)
# print(plotdf.shape)
# display(annodf)
# print(annodf.shape)
# for k in annodf.columns:
#     annodf[k] = 'Antal: '+ annodf[k].astype(int).astype(str)
# for k in annodf2.columns:
#     annodf2[k] = 'Pop: '+ annodf2[k].astype(int).astype(str)
# display(annodf)
# display(annodf2)

# .apply(lambda x: "{:,}".format(x))

for k in annodf3.columns:
    # annodf3[k] = annodf[k].astype(int).astype(str) + ' ud af\n' + annodf2[k].astype(int).astype(str)
    # annodf3_eng[k] = annodf[k].astype(int).astype(str) + ' out of\n' + annodf2[k].astype(int).astype(str)
    annodf3[k] = (annodf[k].astype(int).apply(lambda x: "{:,}".format(x)) + ' ud af\n' + annodf2[k].astype(int).apply(lambda x: "{:,}".format(x))).astype(str).str.replace(',','.')
    annodf3_eng[k] = annodf[k].astype(int).apply(lambda x: "{:,}".format(x)) + ' out of\n' + annodf2[k].astype(int).apply(lambda x: "{:,}".format(x))
annodf3

plotdf_unvacc = pd.DataFrame(fullMatUnvaccInci.transpose())
annodf_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())
annodf2_unvacc = pd.DataFrame(fullMatUnvaccPop.transpose())
annodf3_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())
annodf3_unvacc_eng = pd.DataFrame(fullMatUnvaccCount.transpose())

for k in annodf3_unvacc.columns:
    # annodf3_unvacc[k] = annodf_unvacc[k].astype(int).astype(str) + ' ud af\n' + annodf2_unvacc[k].astype(int).astype(str)
    # annodf3_unvacc_eng[k] = annodf_unvacc[k].astype(int).astype(str) + ' out of\n' + annodf2_unvacc[k].astype(int).astype(str)
    annodf3_unvacc[k] = (annodf_unvacc[k].astype(int).apply(lambda x: "{:0,}".format(x))  + ' ud af\n' + annodf2_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x))).astype(str).str.replace(',','.')
    annodf3_unvacc_eng[k] = annodf_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x))  + ' out of\n' + annodf2_unvacc[k].astype(int).apply(lambda x: "{:,}".format(x)) 
annodf3_unvacc


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3 ud af\n134.544,2 ud af\n120.285,3 ud af\n108.897,3 ud af\n100.863,1 ud af\n95.682,0 ud af\n95.682,0 ud af\n95.682,0 ud af\n88.861,2 ud af\n87.968,0 ud af\n87.968,1 ud af\n86.384,2 ud af\n84.942,3 ud af\n82.653,2 ud af\n78.747,2 ud af\n73.370,2 ud af\n69.615,3 ud af\n66.663,1 ud af\n63.790,5 ud af\n60.687
1,3 ud af\n48.950,6 ud af\n46.571,6 ud af\n44.478,2 ud af\n42.836,0 ud af\n42.836,0 ud af\n42.836,1 ud af\n40.561,0 ud af\n40.385,2 ud af\n40.156,1 ud af\n39.947,3 ud af\n39.714,1 ud af\n39.431,0 ud af\n39.046,2 ud af\n38.183,2 ud af\n36.529,1 ud af\n35.199,5 ud af\n33.582,2 ud af\n32.118,4 ud af\n31.178
2,55 ud af\n402.259,70 ud af\n382.043,57 ud af\n365.771,51 ud af\n354.228,34 ud af\n346.822,13 ud af\n341.185,12 ud af\n337.606,13 ud af\n335.301,23 ud af\n333.584,25 ud af\n331.706,34 ud af\n329.482,43 ud af\n326.269,66 ud af\n321.885,49 ud af\n313.255,63 ud af\n299.745,72 ud af\n289.824,75 ud af\n280.534,80 ud af\n272.015,89 ud af\n265.404
3,39 ud af\n184.796,52 ud af\n179.701,38 ud af\n175.100,47 ud af\n171.610,36 ud af\n169.128,24 ud af\n167.105,12 ud af\n165.713,18 ud af\n164.680,21 ud af\n163.848,30 ud af\n163.041,34 ud af\n162.248,43 ud af\n161.142,51 ud af\n159.674,46 ud af\n156.595,71 ud af\n151.643,71 ud af\n148.275,67 ud af\n145.205,70 ud af\n142.306,89 ud af\n139.926
4,4 ud af\n32.464,6 ud af\n32.056,12 ud af\n31.605,10 ud af\n31.191,6 ud af\n30.914,1 ud af\n30.656,7 ud af\n30.418,6 ud af\n30.197,4 ud af\n30.077,5 ud af\n29.982,7 ud af\n29.868,13 ud af\n29.702,17 ud af\n29.515,16 ud af\n29.178,19 ud af\n28.722,27 ud af\n28.360,23 ud af\n27.987,23 ud af\n27.614,22 ud af\n27.137
5,3 ud af\n10.224,2 ud af\n10.145,6 ud af\n10.045,9 ud af\n9.951,3 ud af\n9.889,0 ud af\n9.812,2 ud af\n9.701,0 ud af\n9.617,1 ud af\n9.560,8 ud af\n9.523,4 ud af\n9.484,8 ud af\n9.430,8 ud af\n9.379,6 ud af\n9.278,13 ud af\n9.141,13 ud af\n9.036,11 ud af\n8.926,14 ud af\n8.799,11 ud af\n8.617


In [19]:
# fig,ax1 = plt.subplots(figsize=(14,8))



# cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

# g = sns.heatmap(data=plotdf,
#                 ax=ax1,
#                 annot=annodf,
#                 annot_kws={'va':'top',"fontsize":7},
#                 fmt='1.0f',
#                 vmax=100,
#                 xticklabels=weekLabelsSpace,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 cbar=False,
#                 )
                

# g.invert_yaxis()
# # plt.yticks(rotation=0)

# # cbar = ax1.collections[0].colorbar
# # cbar.set_ticks(levelsToShow)

# g = sns.heatmap(data=plotdf,
#                 ax=ax1,
#                 annot=plotdf,
#                 annot_kws={'va':'bottom'},
#                 fmt='1.0f',
#                 vmax=100,
#                 xticklabels=weekLabelsSpace,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)

# cbar = ax1.collections[1].colorbar
# cbar.set_ticks(levelsToShow)
# # cbar = ax1.collections[0].colorbar
# # cbar.set_ticks(levelsToShow)
# # fig.axes[2].set_visible(False)

# plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')
# plt.tight_layout()

# ax1.text(18,-1.15,SourceText,ha='right',fontsize=7)

# if saveFigures:
#     plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated_Detailed')

In [20]:
fig,ax1 = plt.subplots(figsize=(14,8))



cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3,
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":13,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)
# fig.axes[2].set_visible(False)

plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')

ax1.text(18,-1.15,SourceText,ha='right',fontsize=7)
plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Vaccinated_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig,ax1 = plt.subplots(figsize=(14,8))



cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax1,
                annot=annodf3_unvacc,
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax1,
                annot=plotdf_unvacc,
                annot_kws={'va':'bottom',"fontsize":13,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)
# fig.axes[2].set_visible(False)

plt.title('Nye indlæggelser per 100.000 uvaccinerede i aldersgruppen')
ax1.text(18,-1.15,SourceText,ha='right',fontsize=7)
plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(18,10))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3,
                annot_kws={'va':'top',"fontsize":6},
                fmt='',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=annodf3_unvacc,
                annot_kws={'va':'top',"fontsize":6},
                fmt='',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=plotdf_unvacc,
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[1].colorbar
cbar.set_ticks(levelsToShow)

# cbar = ax2.collections[1].colorbar
# cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

ax2.text(18,-1.15,SourceText,ha='right',fontsize=7)
fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(18,10))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3_eng,
                annot_kws={'va':'top',"fontsize":6},
                fmt='',
                vmax=100,
                xticklabels=weekLabels_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=annodf3_unvacc_eng,
                annot_kws={'va':'top',"fontsize":6},
                fmt='',
                vmax=100,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax2,
                annot=plotdf_unvacc,
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace_eng,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[1].colorbar
cbar.set_ticks(levelsToShow)

# cbar = ax2.collections[1].colorbar
# cbar.set_ticks(levelsToShow)

# ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
# ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')
ax1.set_title('New hospital-admissions per\n100,000 vaccinated in group')
ax2.set_title('New hospital-admissions per \n100,000 unvaccinated in group')

ax2.text(18,-1.15,SourceText_eng,ha='right',fontsize=7)
fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_English_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plotdf.iloc[:,5:]
firstWeekIndexToShow = 7
weekLabelsSpace_eng[firstWeekIndexToShow:]

['    Week 39',
 '    Week 40',
 '    Week 41',
 '    Week 42',
 '    Week 43',
 '    Week 44',
 '    Week 45',
 '    Week 46',
 '    Week 47',
 '    Week 48',
 '    Week 49',
 '    Week 50']

In [27]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(18,10))

firstWeekIndexToShow = 7

cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf.iloc[:,firstWeekIndexToShow:],
                ax=ax1,
                annot=annodf3_eng.iloc[:,firstWeekIndexToShow:],
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabels_eng[firstWeekIndexToShow:],
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf.iloc[:,firstWeekIndexToShow:],
                ax=ax1,
                annot=plotdf.iloc[:,firstWeekIndexToShow:],
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels_eng[firstWeekIndexToShow:],
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(data=plotdf_unvacc.iloc[:,firstWeekIndexToShow:],
                ax=ax2,
                annot=annodf3_unvacc_eng.iloc[:,firstWeekIndexToShow:],
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabelsSpace_eng[firstWeekIndexToShow:],
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc.iloc[:,firstWeekIndexToShow:],
                ax=ax2,
                annot=plotdf_unvacc.iloc[:,firstWeekIndexToShow:],
                annot_kws={'va':'bottom',"fontsize":14,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabelsSpace_eng[firstWeekIndexToShow:],
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[1].colorbar
cbar.set_ticks(levelsToShow)

# cbar = ax2.collections[1].colorbar
# cbar.set_ticks(levelsToShow)

# ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
# ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')
ax1.set_title('New hospital-admissions per\n100,000 vaccinated in group')
ax2.set_title('New hospital-admissions per \n100,000 unvaccinated in group')

ax2.text(18,-1.15,SourceText_eng,ha='right',fontsize=7)
fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_English_Detailed_Shorter')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Other

In [ ]:
# fullMatVaccInci.transpose()

# plotdf = pd.DataFrame()
# annodf = pd.DataFrame()
# annodf2 = pd.DataFrame()
# for k in range(len(allAgeGroups)):
#     plotdf[allAgeGroups[k]] = fullMatVaccInci.transpose()[k]
#     # annodf[allAgeGroups[k]] = fullMatVaccInci.transpose()[k]



#     thisdf = dfShort[(dfShort.Aldersgruppe == allAgeGroups[k]) & (dfShort.Vaccinationsstatus == 'Forventet fuld effekt')]
#     popSize = thisdf['Befolkningstørrelse'].values
#     numHosp = thisdf['Antal nyindlagte'].values
    
#     # print(popSize)
#     # annodf[allAgeGroups[k]] = popSize
#     # annodf2[allAgeGroups[k]] = numHosp
# display(thisdf)
# plotdf

In [ ]:
thisdfUnvacc = dfUnvacc[dfUnvacc.Uge == '2021-W47']
thisdfVacc = dfVacc[dfVacc.Uge == '2021-W47'] 
thisdfVacc

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
804,2021-W47,12-15,Forventet fuld effekt,1,199075,0.50
807,2021-W47,16-19,Forventet fuld effekt,3,235374,1.27
810,2021-W47,20-39,Forventet fuld effekt,33,1138264,2.90
813,2021-W47,40-64,Forventet fuld effekt,73,1734684,4.21
816,2021-W47,65-79,Forventet fuld effekt,106,860734,12.32
819,2021-W47,80+,Forventet fuld effekt,70,256461,27.29


In [ ]:
totHospUnva = thisdfUnvacc['Antal nyindlagte'].sum()
totHospVacc = thisdfVacc['Antal nyindlagte'].sum()

avgPopUnva = int(np.round(thisdfUnvacc.Befolkningstørrelse.sum()))
avgPopVacc = int(np.round(thisdfVacc.Befolkningstørrelse.sum()))

print(f'{totHospUnva} indlæggelser ud af omkring {avgPopUnva} uvaccinerede')
print(f'{totHospVacc} indlæggelser ud af omkring {avgPopVacc} vaccinerede')

print(totHospUnva/100)
print(avgPopUnva/100)
print(totHospVacc/100)
print(avgPopVacc/100)

186 indlæggelser ud af omkring 580309 uvaccinerede
286 indlæggelser ud af omkring 4424592 vaccinerede
1.86
5803.09
2.86
44245.92


In [ ]:
totHospUnva = agedfUnvacc['Antal nyindlagte'].sum()
totHospVacc = agedfVacc['Antal nyindlagte'].sum()

avgPopUnva = int(np.round(agedfUnvacc.Befolkningstørrelse.mean()))
avgPopVacc = int(np.round(agedfVacc.Befolkningstørrelse.mean()))

print(f'{totHospUnva} indlæggelser ud af omkring {avgPopUnva} uvaccinerede 80+')
print(f'{totHospVacc} indlæggelser ud af omkring {avgPopVacc} vaccinerede 80+')

print(totHospUnva/100)
print(avgPopUnva/100)
print(totHospVacc/100)
print(avgPopVacc/100)

NameError: name 'agedfUnvacc' is not defined

In [ ]:
UH = int(np.round(totHospUnva/100))
U = int(np.round(avgPopUnva/100))
VH = int(np.round(totHospVacc/100))
V = int(np.round(avgPopVacc/100))

# plt.figure()


# m1 = [1,1]
# v1 = [0.4,0.4]
# k1 = V

# data1 = v1 * np.random.randn(k1,2) + m1

rawData = np.random.rand(V-VH,2)

data1 = rawData.copy()
curRadius = 1
data1[:,0] = curRadius* np.sin(2*np.pi*rawData[:,0])*np.sqrt(rawData[:,1])
data1[:,1] = curRadius* np.cos(2*np.pi*rawData[:,0])*np.sqrt(rawData[:,1])

ts = np.linspace(-1,1,3000)
xs = curRadius * np.sin(2*np.pi*ts)
ys = curRadius * np.cos(2*np.pi*ts)

rawData2 = np.random.rand(U-UH,2)

data2 = rawData2.copy()
curRadius2 = 0.25
data2[:,0] = curRadius2* np.sin(2*np.pi*rawData2[:,0])*np.sqrt(rawData2[:,1]) - curRadius - curRadius2
data2[:,1] = curRadius2* np.cos(2*np.pi*rawData2[:,0])*np.sqrt(rawData2[:,1])

# data1[data1[:,0] < -2.75,0] = data1[data1[:,0] < -2.75,0] + 0.2

xs2 = curRadius2 * np.sin(2*np.pi*ts)- curRadius - curRadius2
ys2 = curRadius2 * np.cos(2*np.pi*ts)

plt.figure(figsize=(30,30))
# plt.plot(rawData[:,0],rawData[:,1],'.')
# plt.plot(xs,ys,'m:',linewidth=1)
plt.plot(data1[:,0],data1[:,1],'m.',markersize=15)
plt.plot(data2[:,0],data2[:,1],'g.',markersize=15)

plt.axis('equal')

if saveFigures:
    plt.savefig(path_figs+'HospCircle80+')
    
    
plt.plot(xs,ys,'m:',linewidth=1)
plt.plot(xs2,ys2,'g:',linewidth=1)

if saveFigures:
    plt.savefig(path_figs+'HospCircle80+_Boundary')

# plt.figure()
# # plt.plot(rawData[:,0],rawData[:,1],'.')
# plt.plot(xs,ys,'k',linewidth=1)
# plt.plot(data1[:,0],data1[:,1],'.',markersize=3)
# zoomLevel = 0.2
# plt.xlim([-3-zoomLevel,-3+zoomLevel])
# plt.ylim([-zoomLevel,zoomLevel])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

def circle_points(r, n):
    circles = []
    for r, n in zip(r, n):
        t = np.linspace(0, 2*np.pi, n, endpoint=False)
        x = r * np.cos(t)
        y = r * np.sin(t)
        circles.append(np.c_[x, y])
    return circles

In [ ]:
r = [0, 0.1, 0.2]
n = [1, 10, 20]
circles = circle_points(r, n)
fig, ax = plt.subplots()
for circle in circles:
    ax.scatter(circle[:, 0], circle[:, 1])
ax.set_aspect('equal')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Old stuff and other stuff

In [ ]:

# from matplotlib.colors import ListedColormap
# cmap = ListedColormap(['red','black','yellow', 'blue'])
# # colorbar.set_ticklabels(['A', 'B', 'C', 'NA'])

# fig,ax1 = plt.subplots(figsize=(16,8))
# g = sns.heatmap(fullMatVaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.0f',
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)

In [ ]:


# # curMax = fullMatVaccInci.max()


# levelsToShow = [0,1,5,10,20,50]
# # boundaries = np.array([0.0,5,10,20,50]) / 50
# # # boundaries = [0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]  # custom boundaries
# # # boundaries = np.arange(0,1,10)
# # # hex_colors = sns.light_palette('navy', n_colors=len(boundaries) * 2 + 2, as_cmap=False).as_hex()
# # hex_colors = sns.light_palette('navy', n_colors=len(boundaries) + 2, as_cmap=False).as_hex()
# # hex_colors = [hex_colors[i] for i in range(0, len(hex_colors), 1)]
# # colors=list(zip(boundaries, hex_colors))
# # custom_color_map = LinearSegmentedColormap.from_list(
# #     name='custom_navy',
# #     colors=colors,
# # )

# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) + 1, as_cmap=False).as_hex()
# # myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))


# fig,ax1 = plt.subplots(figsize=(16,8))
# g = sns.heatmap(fullMatVaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.1f',
#                 vmax=50,
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 # cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8, 1.0]}
#                 )
# g.invert_yaxis()

# # colorbar = ax1.collections[0].colorbar
# # colorbar.set_ticks(levelsToShow)
# # colorbar.set_ticklabels(['B', 'A', 'C'])
# plt.yticks(rotation=0)

In [ ]:

# curdf = df 
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Ikke vaccineret')]
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Forventet fuld effekt')]
curdf

curWeekNum = curdf.Uge.apply(lambda x: int(x[6:]))

curAge = '20-39'
for curAge in dfShort.Aldersgruppe.unique():
    curAgeLabel = 'Personer i aldersgruppen '+curAge
    agedf = dfShort[(dfShort.Aldersgruppe == curAge)]

    curSums = agedf.groupby('Uge').sum()

    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


    # ax1.plot(curWeekNum,curSums['Antal nyindlagte'])

    agedfVacc = agedf[agedf.Vaccinationsstatus == 'Forventet fuld effekt']
    agedfUnvacc = agedf[agedf.Vaccinationsstatus == 'Ikke vaccineret']
    # ax1.fill_between(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.fill_between(curWeekNum,agedfVacc['Antal nyindlagte'].values,agedfUnvacc['Antal nyindlagte'].values,color='b')

    curWeekNum = agedfVacc.Uge.apply(lambda x: int(x[6:]))

    hospUn = agedfUnvacc['Antal nyindlagte'].values
    hospVa = agedfVacc['Antal nyindlagte'].values
    hospTot = hospUn + hospVa

    # ax1.plot(curWeekNum,hospUn)
    # ax1.plot(curWeekNum,hospTot,'k')
    ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
    ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')
    # ax1.plot(rnTime(curWeekNum,5),rnMean(100*hospUn/hospTot,5),color='r')
    # ax1.plot(curWeekNum,hospVa/hospTot,color='b')
    # ax1.fill_between(curWeekNum,hospVa/hospTot,hospUn/hospTot,color='b')
    # ax2.fill_between(curWeekNum,agedfUnvacc['Befolkningstørrelse'].values,color='xkcd:light red')
    # ax2.fill_between(curWeekNum,agedfVacc['Befolkningstørrelse'].values,agedfUnvacc['Befolkningstørrelse'].values,color='b')

    popUn = agedfUnvacc['Befolkningstørrelse'].values
    popVa = agedfVacc['Befolkningstørrelse'].values
    popTot = popUn + popVa
    ax2.fill_between(curWeekNum,100*popTot/popTot,color='b')
    ax2.fill_between(curWeekNum,100*popUn/popTot,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfVacc['Antal nyindlagte'].values,color='b')
    # ax1.stackplot(curWeekNum,
    #                 agedfUnvacc['Antal nyindlagte'],
    #                 agedfVacc['Antal nyindlagte'])

    # plt.plot(curWeekNum,curdf['Nyindlagte pr. 100.000'])
    # # plt.plot(curWeekNum,curdf['Befolkningstørrelse'])

    ax1.grid(axis='y')
    ax2.grid(axis='y')
    ax1.set_ylabel('Andel af alle indlæggelser [%]')
    ax2.set_ylabel('Andel af befolkningsstørrelse [%]')

    ax1.set_xlim([curWeekNum.values[0],curWeekNum.values[-1]])
    ax1.set_title(curAgeLabel)

    curMax = 100
    ax1.set_ylim(bottom=0,top=curMax)
    ax2.set_ylim(bottom=0,top=curMax)
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-41-c3c13671c328>:33: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
<ipython-input-41-c3c13671c328>:34: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

# curdf = df 
curdf = shortdf[(shortdf.Aldersgruppe == '80+') & (shortdf.Vaccinationsstatus == 'Ikke vaccineret')]
curdf

plt.figure()

# plt.plot(curdf.)

NameError: name 'shortdf' is not defined